<a href="https://colab.research.google.com/github/mrpintime/Santander-Customer-Transaction/blob/main/SantanderCustomerTransaction(Kaggle_competition).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Customer Specific Transactions
Created by Moein aka Mrpintime

# **DataSet Description**:
`name`: SantanderCustomerSatisfaction
`version`: 3
`Author`: Banco Santander
`description`:
At Santander our mission is to help people and businesses prosper. We are always looking for ways to help our customers understand their financial health and identify which products and services might help them achieve their monetary goals.

Our data science team is continually challenging our machine learning algorithms, working with the global data science community to make sure we can more accurately identify new ways to solve our most common challenge, binary classification problems such as: is a customer satisfied? Will a customer buy this product? Can a customer pay this loan?

**Dataset taken from Kaggle**: https://www.kaggle.com/c/santander-customer-transaction-prediction/data


---


# Problem


In this challenge, we want to identify which customers will make a specific transaction in the future, irrespective of the amount of money transacted.
**Note**: The data provided for this competition has the same structure as the real data we have available to solve this problem.

# Import files from Kaggle

In [ ]:
# !pip install opendatasets
# import opendatasets as op
# op.download("https://www.kaggle.com/competitions/santander-customer-transaction-prediction/data")

Install cuML and lightgbm for faster computing

In [ ]:
# this cell is for install cuML on Colab
# !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
# !python rapidsai-csp-utils/colab/pip-install.py
# install LightGBM
# !pip install lightgbm

# Pre-processing (Cleansing)

Import necessary libraries

In [ ]:
import pandas as pd, seaborn as sns, numpy as np, matplotlib.pyplot as plt

In [ ]:
# save files to google drive
# ! cp -r "/content/santander-customer-transaction-prediction/" "/content/drive/MyDrive/"

Read train set

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/santander-customer-transaction-prediction/train.csv")
df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


**Note**: We have a very large dataset

In [ ]:
df.shape

(200000, 202)

Check the null values, feature variables, target variable and clean them

In [ ]:
df.isnull().sum()

ID_code    0
target     0
var_0      0
var_1      0
var_2      0
          ..
var_195    0
var_196    0
var_197    0
var_198    0
var_199    0
Length: 202, dtype: int64

In [ ]:
df.isnull().sum().sum()

0

In [ ]:
df.columns

Index(['ID_code', 'target', 'var_0', 'var_1', 'var_2', 'var_3', 'var_4',
       'var_5', 'var_6', 'var_7',
       ...
       'var_190', 'var_191', 'var_192', 'var_193', 'var_194', 'var_195',
       'var_196', 'var_197', 'var_198', 'var_199'],
      dtype='object', length=202)

In [ ]:
df.dtypes

ID_code     object
target       int64
var_0      float64
var_1      float64
var_2      float64
            ...   
var_195    float64
var_196    float64
var_197    float64
var_198    float64
var_199    float64
Length: 202, dtype: object

In [ ]:
df.dtypes.unique()

array([dtype('O'), dtype('int64'), dtype('float64')], dtype=object)

In [ ]:
df = df.drop(columns='ID_code')

Descriptive statistics

In [ ]:
df.describe()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,0.100490,10.679914,-1.627622,10.715192,6.796529,11.078333,-5.065317,5.408949,16.545850,0.284162,...,3.234440,7.438408,1.927839,3.331774,17.993784,-0.142088,2.303335,8.908158,15.870720,-3.326537
std,0.300653,3.040051,4.050044,2.640894,2.043319,1.623150,7.863267,0.866607,3.418076,3.332634,...,4.559922,3.023272,1.478423,3.992030,3.135162,1.429372,5.454369,0.921625,3.010945,10.438015
min,0.000000,0.408400,-15.043400,2.117100,-0.040200,5.074800,-32.562600,2.347300,5.349700,-10.505500,...,-14.093300,-2.691700,-3.814500,-11.783400,8.694400,-5.261000,-14.209600,5.960600,6.299300,-38.852800
25%,0.000000,8.453850,-4.740025,8.722475,5.254075,9.883175,-11.200350,4.767700,13.943800,-2.317800,...,-0.058825,5.157400,0.889775,0.584600,15.629800,-1.170700,-1.946925,8.252800,13.829700,-11.208475
50%,0.000000,10.524750,-1.608050,10.580000,6.825000,11.108250,-4.833150,5.385100,16.456800,0.393700,...,3.203600,7.347750,1.901300,3.396350,17.957950,-0.172700,2.408900,8.888200,15.934050,-2.819550
75%,0.000000,12.758200,1.358625,12.516700,8.324100,12.261125,0.924800,6.003000,19.102900,2.937900,...,6.406200,9.512525,2.949500,6.205800,20.396525,0.829600,6.556725,9.593300,18.064725,4.836800
max,1.000000,20.315000,10.376800,19.353000,13.188300,16.671400,17.251600,8.447700,27.691800,10.151300,...,18.440900,16.716500,8.402400,18.281800,27.928800,4.272900,18.321500,12.000400,26.079100,28.500700


We have very imbalabce dataset.

In [ ]:
df.target.value_counts(normalize=1)

0    0.89951
1    0.10049
Name: target, dtype: float64

we can do several things for this:
- Class weight approache
- Downsampling
- Upsampling
- Resampling

Import necessary library for imbalance dataset.

In [ ]:
from imblearn.under_sampling import RandomUnderSampler, NearMiss, TomekLinks
from imblearn.over_sampling import RandomOverSampler, KMeansSMOTE, SMOTE, SMOTEN

Our data set is clean but first we want to analyse it without Dimension reduction technique like `PCA` or `Clustering` ...
but you have to know your data set so good, because when you do Dimension Reduction technique you lose some information



---



# Data preprocessing for predictive analysis

import necessary libraries

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.iloc[:, 1:]
y = df.target

X.shape, y.shape

((200000, 200), (200000,))

**Note:** in kaggle competitions ussually we have 2 dataset one for train and another one for test so we can train our model on entire train dataset after validate our model.
and another thing is that you have to check that do you have same values and features in train and test set or there are some differences.

Let's our data set into `train` and `validate`.

`Tip`: If you get enough accuracy in ypu model, you can improve it by `Psudo labeling technique` but be aware about **overfitting**.

we use `stratify` parametere here because we want to save ferquency of target in train and validate sets

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y, test_size=0.3, random_state=123)

As we can see validate and train sets have same relative ferquencies for their values but they are very `imbalance`.

In [ ]:
y_train.value_counts(normalize=1), y_valid.value_counts(normalize=1)

(0    0.899507
 1    0.100493
 Name: target, dtype: float64,
 0    0.899517
 1    0.100483
 Name: target, dtype: float64)

## data sampling for imbalance dataset

Because we have a big dataset and we do not want to create instances that are not 100 pecent real so we decide to do downsampling first.

### Downsampling

#### RandomUnderSampler, NearMiss, TomekLinks

In [ ]:
# Implement down sampling techniques.